In [4]:
# build_valuation_sequence_rnn_npz.py
#
# Builds an NPZ where the *sequence timesteps are valuation dates* (not games).
# Each timestep summarizes what happened between two valuation dates.
#
# Output NPZ keys:
#   - X_seq:      (N, T, F_seq) float32   sequence of valuation-interval features
#   - X_static:   (N, F_static) float32   static player features (height + OHE foot/pos)
#   - y:          (N,) float32            target = y_log at target valuation date
#   - player_id:  (N,) int64
#   - target_date:(N,) datetime64[ns]
#
# T = number of past valuation-steps used to predict the next valuation.

import numpy as np
import pandas as pd
from pathlib import Path

# ----------------------------
# Paths (portable)
# ----------------------------
try:
    PROJECT_ROOT = Path(__file__).resolve().parent.parent
except NameError:
    PROJECT_ROOT = Path.cwd().parent  # if running from a notebook folder

DATA_DIR = PROJECT_ROOT / "Data"
OUT_DIR = PROJECT_ROOT / "Data_Processed"
OUT_DIR.mkdir(parents=True, exist_ok=True)

PLAYERS_CSV = DATA_DIR / "players.csv"
VALUATIONS_CSV = DATA_DIR / "player_valuations.csv"
EVENTS_CSV = DATA_DIR / "game_events.csv"

OUT_NPZ = OUT_DIR / "valuation_seq_rnn_dataset.npz"

# ----------------------------
# Config
# ----------------------------
T = 5                    # sequence length in valuation-steps
MIN_VALUATIONS = T + 1   # need at least T+1 valuation dates to form 1 sample
USE_LOG_TARGET = True    # log1p target
CLIP_H_DAYS = True
H_DAYS_MAX = 3650        # cap 10 years (optional)

# ----------------------------
# Helpers
# ----------------------------
def safe_to_datetime(s):
    return pd.to_datetime(s, errors="coerce", utc=False)

def to_int_player_id(df, col="player_id"):
    df[col] = pd.to_numeric(df[col], errors="coerce")
    df = df.dropna(subset=[col]).copy()
    df[col] = df[col].astype(np.int64)
    return df

def standardize_position(pos):
    if pd.isna(pos):
        return "UNK"
    p = str(pos).upper()
    if "GOAL" in p or p == "GK":
        return "GK"
    if "DEF" in p:
        return "DEF"
    if "MID" in p:
        return "MID"
    if "ATT" in p or "FORW" in p or "WING" in p or "STRIK" in p:
        return "ATT"
    return p[:10]

def standardize_foot(foot):
    if pd.isna(foot):
        return "UNK"
    f = str(foot).lower()
    if f.startswith("right"):
        return "R"
    if f.startswith("left"):
        return "L"
    if "both" in f:
        return "B"
    return "UNK"

def compute_age_years(dob, ref_date):
    if pd.isna(dob) or pd.isna(ref_date):
        return np.nan
    return (ref_date - dob).days / 365.25

def make_big5_flag(val_df):
    # England, Spain, Italy, Germany, France (common Transfermarkt IDs)
    BIG5_IDS = {"GB1", "ES1", "IT1", "DE1", "FR1"}
    if "player_club_domestic_competition_id" not in val_df.columns:
        val_df["is_big5_league"] = 0.0
        return val_df
    comp = val_df["player_club_domestic_competition_id"].fillna("").astype(str).str.upper()
    val_df["is_big5_league"] = comp.isin(BIG5_IDS).astype(np.float32)
    return val_df

# ----------------------------
# Load CSVs
# ----------------------------
print("Loading CSVs...")
players = pd.read_csv(PLAYERS_CSV, low_memory=False)
valuations = pd.read_csv(VALUATIONS_CSV, low_memory=False)
events = pd.read_csv(EVENTS_CSV, low_memory=False)

players = to_int_player_id(players, "player_id")
valuations = to_int_player_id(valuations, "player_id")

players["date_of_birth"] = safe_to_datetime(players.get("date_of_birth"))
valuations["date"] = safe_to_datetime(valuations.get("date"))

events["date"] = safe_to_datetime(events.get("date"))
events["game_id"] = pd.to_numeric(events.get("game_id"), errors="coerce")
events = events.dropna(subset=["date", "game_id"]).copy()
events["game_id"] = events["game_id"].astype(np.int64)

# Make sure event player columns are numeric where present
for c in ["player_id", "player_assist_id", "player_in_id"]:
    if c in events.columns:
        events[c] = pd.to_numeric(events[c], errors="coerce")

# Clean valuations
valuations = valuations.dropna(subset=["date", "market_value_in_eur"]).copy()
valuations["market_value_in_eur"] = pd.to_numeric(valuations["market_value_in_eur"], errors="coerce")
valuations = valuations.dropna(subset=["market_value_in_eur"]).copy()
valuations = valuations.sort_values(["player_id", "date"]).reset_index(drop=True)

# ----------------------------
# Build static player features (X_static)
# ----------------------------
print("Building static player features...")

pstat = players[["player_id", "height_in_cm", "foot", "position", "date_of_birth"]].copy()
pstat["height_in_cm"] = pd.to_numeric(pstat["height_in_cm"], errors="coerce")
pstat["foot"] = pstat["foot"].apply(standardize_foot)
pstat["pos_group"] = pstat["position"].apply(standardize_position)

static_ohe = pd.get_dummies(
    pstat[["foot", "pos_group"]].fillna("UNK"),
    prefix=["foot", "pos"],
)

pstat_num = pd.concat(
    [
        pstat[["player_id", "height_in_cm", "date_of_birth"]].reset_index(drop=True),
        static_ohe.reset_index(drop=True),
    ],
    axis=1,
).drop_duplicates("player_id")

static_cols = ["height_in_cm"] + [c for c in pstat_num.columns if c.startswith("foot_") or c.startswith("pos_")]
pstat_num[static_cols] = pstat_num[static_cols].fillna(0.0)

# Make a lookup dict for speed
pstat_lookup = {}
for _, row in pstat_num.iterrows():
    pid = int(row["player_id"])
    pstat_lookup[pid] = (
        float(row.get("height_in_cm", 0.0)),
        row[static_cols[1:]].to_numpy(dtype=np.float32),  # OHE part
        row.get("date_of_birth", pd.NaT),
    )

# ----------------------------
# Build per-game event counts per player (for interval aggregation)
# ----------------------------
print("Building per-game event features...")

desc = events.get("description", pd.Series([""] * len(events))).fillna("")
etype = events.get("type", pd.Series([""] * len(events)))

is_goal = etype == "Goals"
is_yellow = (etype == "Cards") & desc.str.contains("Yellow card", case=False, na=False)
is_red = (etype == "Cards") & desc.str.contains("Red card", case=False, na=False)
is_sub = etype == "Substitutions"

def count_events(df, col, name):
    if col not in df.columns:
        return pd.DataFrame(columns=["player_id", "game_id", name])
    tmp = df[[col, "game_id"]].dropna().copy()
    tmp[col] = pd.to_numeric(tmp[col], errors="coerce")
    tmp = tmp.dropna(subset=[col])
    tmp[col] = tmp[col].astype(np.int64)
    out = (
        tmp.groupby([col, "game_id"])
        .size()
        .rename(name)
        .reset_index()
        .rename(columns={col: "player_id"})
    )
    return out

goals = count_events(events[is_goal], "player_id", "goals")
assists = count_events(events[is_goal], "player_assist_id", "assists")
yellow = count_events(events[is_yellow], "player_id", "yellow_cards")
red = count_events(events[is_red], "player_id", "red_cards")
sub_in = count_events(events[is_sub], "player_in_id", "sub_in")
sub_out = count_events(events[is_sub], "player_id", "sub_out")

game_dates = events.groupby("game_id")["date"].min().reset_index(name="game_date")

pairs = pd.concat([goals, assists, yellow, red, sub_in, sub_out], axis=0)[["player_id", "game_id"]]
pairs = pairs.dropna().drop_duplicates()

per_game = pairs.merge(game_dates, on="game_id", how="left")

for df in [goals, assists, yellow, red, sub_in, sub_out]:
    per_game = per_game.merge(df, on=["player_id", "game_id"], how="left")

per_game = per_game.fillna(0.0)
per_game = per_game.dropna(subset=["game_date"]).copy()
per_game = per_game.sort_values(["player_id", "game_date"]).reset_index(drop=True)

GAME_FEATURES = ["goals", "assists", "yellow_cards", "red_cards", "sub_in", "sub_out"]

# Group per player for aggregation
pgroups = {pid: g.reset_index(drop=True) for pid, g in per_game.groupby("player_id")}

# ----------------------------
# Prepare valuations with useful columns
# ----------------------------
print("Preparing valuations...")
val = valuations.copy()
val = make_big5_flag(val)

val["y_raw"] = val["market_value_in_eur"].astype(np.float32)
val["y_log"] = np.log1p(val["y_raw"]).astype(np.float32)

# We'll compute age at valuation time using DOB from players
# Build valuation groups per player
vgroups = {pid: g.sort_values("date").reset_index(drop=True) for pid, g in val.groupby("player_id")}

# ----------------------------
# Build valuation-step sequences
# ----------------------------
print("Building valuation-step sequences...")

X_seq_list = []
X_static_list = []
y_list = []
pid_list = []
target_date_list = []

# Each timestep feature vector (F_seq):
# [prev_y_log, H_days, num_games, goals_sum, assists_sum, yellow_sum, red_sum, sub_in_sum, sub_out_sum, age_prev, big5_prev]
# = 1 + 1 + 1 + 6 + 1 + 1 = 11
F_SEQ = 11

def get_interval_sums_for_player(pid, start_date, end_date):
    """
    Sum GAME_FEATURES for games with (start_date, end_date] for a given player.
    Uses cumulative sums for speed.
    """
    g = pgroups.get(pid)
    if g is None or len(g) == 0:
        return 0, np.zeros(len(GAME_FEATURES), dtype=np.float32)

    g_dates = g["game_date"].to_numpy()
    feats = g[GAME_FEATURES].to_numpy(dtype=np.float32)

    # cumulative sums (prefix)
    csum = np.cumsum(feats, axis=0)
    # use side='right' to include games on the same date as end_date
    idx_end = np.searchsorted(g_dates, np.datetime64(end_date), side="right")
    idx_start = np.searchsorted(g_dates, np.datetime64(start_date), side="right")

    num_games = int(max(0, idx_end - idx_start))
    if num_games == 0:
        return 0, np.zeros(len(GAME_FEATURES), dtype=np.float32)

    end_vec = csum[idx_end - 1]
    start_vec = csum[idx_start - 1] if idx_start > 0 else np.zeros(len(GAME_FEATURES), dtype=np.float32)
    sums = (end_vec - start_vec).astype(np.float32)
    return num_games, sums

n_players_used = 0
n_samples = 0

for pid, vg in vgroups.items():
    if pid not in pstat_lookup:
        continue
    if pid not in pgroups:
        continue

    if len(vg) < MIN_VALUATIONS:
        continue

    height, ohe_vec, dob = pstat_lookup[pid]
    # Build a list of valuation-step input vectors for this player
    # step i predicts y_i from info at i-1 and games (t_{i-1}, t_i]
    step_feats = []
    step_targets = []
    step_target_dates = []

    for i in range(1, len(vg)):
        d_prev = vg.loc[i - 1, "date"]
        d_curr = vg.loc[i, "date"]

        y_prev_log = float(vg.loc[i - 1, "y_log"])
        y_curr_log = float(vg.loc[i, "y_log"])

        H_days = (d_curr - d_prev).days
        if pd.isna(H_days):
            continue
        H_days = int(H_days)
        if H_days <= 0:
            # skip weird duplicates/out-of-order
            continue
        if CLIP_H_DAYS:
            H_days = int(min(H_days, H_DAYS_MAX))

        big5_prev = float(vg.loc[i - 1, "is_big5_league"])

        age_prev = compute_age_years(dob, d_prev)
        if pd.isna(age_prev):
            age_prev = 0.0

        num_games, sums = get_interval_sums_for_player(pid, d_prev, d_curr)

        feat_vec = np.zeros(F_SEQ, dtype=np.float32)
        feat_vec[0] = y_prev_log
        feat_vec[1] = float(H_days)
        feat_vec[2] = float(num_games)
        feat_vec[3:3+len(GAME_FEATURES)] = sums
        feat_vec[9] = float(age_prev)
        feat_vec[10] = float(big5_prev)

        step_feats.append(feat_vec)
        step_targets.append(y_curr_log if USE_LOG_TARGET else float(vg.loc[i, "y_raw"]))
        step_target_dates.append(d_curr)

    if len(step_feats) < T:
        continue

    step_feats = np.asarray(step_feats, dtype=np.float32)
    step_targets = np.asarray(step_targets, dtype=np.float32)

    # Static vector: [height] + OHE foot/pos
    static_vec = np.concatenate([[np.float32(height)], ohe_vec.astype(np.float32)], axis=0).astype(np.float32)

    # Create samples: for each target index k, take last T steps ending at k
    # step_feats[k] predicts step_targets[k]
    for k in range(T - 1, len(step_feats)):
        seq = step_feats[k - (T - 1) : k + 1]  # shape (T, F_SEQ)
        y_tgt = step_targets[k]
        tgt_date = step_target_dates[k]

        X_seq_list.append(seq)
        X_static_list.append(static_vec)
        y_list.append(y_tgt)
        pid_list.append(pid)
        target_date_list.append(tgt_date)
        n_samples += 1

    n_players_used += 1

X_seq = np.asarray(X_seq_list, dtype=np.float32)
X_static = np.asarray(X_static_list, dtype=np.float32)
y_out = np.asarray(y_list, dtype=np.float32)

player_id_arr = np.asarray(pid_list, dtype=np.int64)
target_date_arr = np.asarray(pd.to_datetime(target_date_list).to_numpy(), dtype="datetime64[ns]")

print("Players used:", n_players_used)
print("Samples:", n_samples)
print("X_seq:", X_seq.shape, "X_static:", X_static.shape, "y:", y_out.shape)

# Final safety: remove any non-finite
good = (
    np.isfinite(X_seq).all(axis=(1,2)) &
    np.isfinite(X_static).all(axis=1) &
    np.isfinite(y_out)
)
if (~good).sum() > 0:
    print("Dropping non-finite samples:", int((~good).sum()))
    X_seq = X_seq[good]
    X_static = X_static[good]
    y_out = y_out[good]
    player_id_arr = player_id_arr[good]
    target_date_arr = target_date_arr[good]

print("Final:", X_seq.shape, X_static.shape, y_out.shape)

# ----------------------------
# Save NPZ
# ----------------------------
np.savez_compressed(
    OUT_NPZ,
    X_seq=X_seq,
    X_static=X_static,
    y=y_out,
    player_id=player_id_arr,
    target_date=target_date_arr,
)

print("Saved:", OUT_NPZ)
print("Seq features (F_SEQ):", F_SEQ)
print("Static features:", X_static.shape[1])


Loading CSVs...
Building static player features...
Building per-game event features...
Preparing valuations...
Building valuation-step sequences...
Players used: 23700
Samples: 337430
X_seq: (337430, 5, 11) X_static: (337430, 10) y: (337430,)
Final: (337430, 5, 11) (337430, 10) (337430,)
Saved: c:\Users\johan\OneDrive\IN2010\FYSSTK3155\PROJECT 3\Code\Data_Processed\valuation_seq_rnn_dataset.npz
Seq features (F_SEQ): 11
Static features: 10
